In [1]:
import mysql.connector
from mysql.connector import Error
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
from PIL import Image, ImageTk
db_connection = None

def connect_to_database(username, password):
    global db_connection
    try:
        db_connection = mysql.connector.connect(
            host='localhost',
            database='inventory_management',
            user=username,
            password=password
        )
        if db_connection.is_connected():
            messagebox.showinfo("Success", "Connected to the database successfully!")
            show_main_window()
    except Error as e:
        messagebox.showerror("Error", f"Failed to connect to the database: {e}")

def create_record():
    try:
        table = simpledialog.askstring("Input", "Enter the table name:")
        if not table:
            return
        cursor = db_connection.cursor()
        cursor.execute(f"DESCRIBE {table}")
        columns = cursor.fetchall()
        values = []
        for column in columns:
            column_name = column[0]
            value = simpledialog.askstring("Input", f"Enter value for {column_name}:")
            values.append(value)
        placeholders = ", ".join(["%s"] * len(values))
        columns_list = ", ".join([col[0] for col in columns])
        query = f"INSERT INTO {table} ({columns_list}) VALUES ({placeholders})"
        cursor.execute(query, values)
        db_connection.commit()
        messagebox.showinfo("Success", "Record created successfully!")
    except Error as e:
        messagebox.showerror("Error", f"Failed to create record: {e}")

def read_records():
    try:
        table = simpledialog.askstring("Input", "Enter the table name to read from:")
        if not table:
            return
        cursor = db_connection.cursor()
        cursor.execute(f"SELECT * FROM {table}")
        records = cursor.fetchall()
        display_results(records, f"Records from {table}")
    except Error as e:
        messagebox.showerror("Error", f"Failed to read records: {e}")

def update_record():
    try:
        # First ask for table name
        table = simpledialog.askstring("Input", "Enter the table name to update:")
        if not table:
            return
            
        # Handle reserved words in MySQL
        if table.lower() == "order":
            table = "`order`"  # Escape the reserved word
            
        # Get the primary key field for the selected table
        cursor = db_connection.cursor()
        cursor.execute(f"SHOW KEYS FROM {table} WHERE Key_name = 'PRIMARY'")
        primary_key = cursor.fetchone()[4]  # Get the primary key column name
        
        # Show existing records first
        cursor.execute(f"SELECT * FROM {table}")
        records = cursor.fetchall()
        
        if not records:
            messagebox.showinfo("Info", "No records found in this table")
            return
            
        # Show records in a message box
        record_list = "\n".join([str(record) for record in records])
        messagebox.showinfo("Existing Records", f"Available records:\n{record_list}")
        
        # Get the ID of the record to update
        id_value = simpledialog.askstring("Input", f"Enter {primary_key} of record to update:")
        if not id_value:
            return
            
        # Get the column to update
        cursor.execute(f"DESCRIBE {table}")
        columns = [row[0] for row in cursor.fetchall()]
        column_list = "\n".join(columns)
        column_to_update = simpledialog.askstring("Input", 
            f"Enter column name to update:\nAvailable columns:\n{column_list}")
        
        if not column_to_update or column_to_update not in columns:
            messagebox.showerror("Error", "Invalid column name")
            return
            
        # Get the new value
        new_value = simpledialog.askstring("Input", f"Enter new value for {column_to_update}:")
        if new_value is None:  # User clicked Cancel
            return
            
        # Construct and execute the update query
        query = f"UPDATE {table} SET {column_to_update} = %s WHERE {primary_key} = %s"
        cursor.execute(query, (new_value, id_value))
        db_connection.commit()
        
        if cursor.rowcount > 0:
            messagebox.showinfo("Success", "Record updated successfully!")
        else:
            messagebox.showinfo("Info", "No matching record found to update")
            
    except Error as e:
        if "foreign key constraint fails" in str(e):
            messagebox.showerror("Error", 
                "Cannot update this record because it would violate referential integrity.\n"
                "The new value must exist in the referenced table.")
        else:
            messagebox.showerror("Error", f"Failed to update record: {e}")
        print(f"Error details: {e}")  # For debugging
def delete_record():
    try:
        # First ask for table name
        table = simpledialog.askstring("Input", "Enter the table name to delete from:")
        if not table:
            return
            
        # Handle reserved words in MySQL
        if table.lower() == "order":
            table = "`order`"  # Escape the reserved word
            
        # Get the primary key field for the selected table
        cursor = db_connection.cursor()
        cursor.execute(f"SHOW KEYS FROM {table} WHERE Key_name = 'PRIMARY'")
        primary_key = cursor.fetchone()[4]  # Get the primary key column name
        
        # First show existing records
        cursor.execute(f"SELECT * FROM {table}")
        records = cursor.fetchall()
        
        if not records:
            messagebox.showinfo("Info", "No records found in this table")
            return
            
        # Show records in a message box
        record_list = "\n".join([str(record) for record in records])
        messagebox.showinfo("Existing Records", f"Available records:\n{record_list}")
        
        # Ask for the ID value
        id_value = simpledialog.askstring("Input", f"Enter {primary_key} value to delete:")
        if not id_value:
            return
            
        # Check for foreign key constraints
        if table.lower() in ['product', 'customer', 'sale']:
            # Show warning about foreign key constraints
            warning = messagebox.askyesno("Warning", 
                "Deleting this record might fail if it's referenced by other tables.\n"
                "Do you want to continue?")
            if not warning:
                return
        
        # Construct and execute the delete query
        query = f"DELETE FROM {table} WHERE {primary_key} = %s"
        cursor.execute(query, (id_value,))
        db_connection.commit()
        
        if cursor.rowcount > 0:
            messagebox.showinfo("Success", "Record deleted successfully!")
        else:
            messagebox.showinfo("Info", "No record found with the given ID")
            
    except Error as e:
        if "foreign key constraint fails" in str(e):
            messagebox.showerror("Error", 
                "Cannot delete this record because it is referenced by other tables.\n"
                "You must first delete related records from dependent tables.")
        else:
            messagebox.showerror("Error", f"Failed to delete record: {e}")
        print(f"Error details: {e}")  # For debugging

def top_selling_products():
    query = """
    SELECT p.PID, p.Pname, SUM(sd.SDquantity) AS TotalSold
    FROM Product p
    JOIN Sale_Details sd ON p.PID = sd.PID
    GROUP BY p.PID, p.Pname
    ORDER BY TotalSold DESC
    LIMIT 5
    """
    execute_and_display(query, "Top 5 Best-Selling Products")
def monthly_sales_trend():
    try:
        # First check if there's data
        check_query = """
        SELECT COUNT(*) 
        FROM Sale 
        WHERE sale_date IS NOT NULL AND Stotal_amount IS NOT NULL;
        """
        cursor = db_connection.cursor()
        cursor.execute(check_query)
        count = cursor.fetchone()[0]
        
        if count == 0:
            messagebox.showinfo("Info", "No sales data available. Please insert some sales records first.")
            return

        query = """
        SELECT 
            DATE_FORMAT(sale_date, '%Y-%m') AS Month,
            COALESCE(SUM(Stotal_amount), 0) AS MonthlyTotal,
            COALESCE(SUM(SUM(Stotal_amount)) OVER (ORDER BY DATE_FORMAT(sale_date, '%Y-%m')), 0) AS RunningTotal
        FROM Sale
        WHERE sale_date IS NOT NULL 
        GROUP BY Month
        ORDER BY Month;
        """
        
        cursor.execute(query)
        results = cursor.fetchall()
        
        if not results:
            messagebox.showinfo("Info", "No sales data available")
            return

        # Create result window
        result_window = tk.Toplevel()
        result_window.title("Monthly Sales Trend")
        result_window.geometry("800x600")

        # Create frame for the treeview
        frame = tk.Frame(result_window)
        frame.pack(fill='both', expand=True, padx=10, pady=10)

        # Create the treeview
        columns = ["Month", "Monthly Total", "Running Total"]
        tree = ttk.Treeview(frame, columns=columns, show="headings")
        
        # Set up the columns
        for col in columns:
            tree.heading(col, text=col)
            tree.column(col, width=150)

        # Insert the data
        for row in results:
            # Format the monetary values
            formatted_row = (
                row[0],
                f"${row[1]:,.2f}",
                f"${row[2]:,.2f}"
            )
            tree.insert("", "end", values=formatted_row)

        # Add scrollbar
        scrollbar = ttk.Scrollbar(frame, orient="vertical", command=tree.yview)
        tree.configure(yscrollcommand=scrollbar.set)

        # Pack the treeview and scrollbar
        tree.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        # Add close button
        close_button = tk.Button(
            result_window,
            text="Close",
            command=result_window.destroy,
            bg='#d9534f',
            fg='black',
            font=("Arial", 12)
        )
        close_button.pack(pady=10)

    except Exception as e:
        messagebox.showerror("Error", f"Failed to retrieve sales data: {e}")
        print(f"Error details: {e}")

def product_reorder_alert():
    query = """
    SELECT p.PID, p.Pname, p.Pstock_quantity,
        CASE
            WHEN p.Pstock_quantity < 10 THEN 'Urgent Reorder'
            WHEN p.Pstock_quantity < 50 THEN 'Reorder Soon'
            ELSE 'Stock Sufficient'
        END AS ReorderStatus
    FROM Product p
    ORDER BY p.Pstock_quantity
    """
    execute_and_display(query, "Product Reorder Alert")

def customer_ranking():
    query = """
    SELECT c.CID, c.Cname, 
        SUM(s.Stotal_amount) AS TotalPurchases,
        RANK() OVER (ORDER BY SUM(s.Stotal_amount) DESC) AS CustomerRank
    FROM Customer c
    JOIN Sale s ON c.CID = s.CID
    GROUP BY c.CID, c.Cname
    """
    execute_and_display(query, "Customer Ranking")

def category_sales_analysis():
    query = """
    SELECT 
        p.Pcategory, 
        QUARTER(s.sale_date) AS Quarter,
        SUM(sd.SDquantity * p.Punit_price) AS TotalSales
    FROM Product p
    JOIN Sale_Details sd ON p.PID = sd.PID
    JOIN Sale s ON sd.SID = s.SID
    GROUP BY p.Pcategory, Quarter WITH ROLLUP
    """
    execute_and_display(query, "Product Category Sales Analysis")

def execute_query(query):
    try:
        cursor = db_connection.cursor()
        cursor.execute(query)
        return cursor.fetchall()
    except Error as e:
        messagebox.showerror("Error", f"Failed to execute query: {e}")
        return None

def execute_and_display(query, title):
    results = execute_query(query)
    if results:
        display_results(results, title)
def display_results(results, title):
    result_window = tk.Toplevel()
    result_window.title(title)
    result_window.geometry("800x600")
    result_window.configure(bg="#FFFFFF")  # White background
    
    # Create frame
    frame = tk.Frame(result_window, bg="#FFFFFF")
    frame.pack(fill='both', expand=True, padx=10, pady=10)
    
    # Create Treeview
    tree = ttk.Treeview(frame)
    
    # Configure style
    style = ttk.Style()
    style.configure("Treeview",
        background="#FFFFFF",
        foreground="#2C3E50",
        fieldbackground="#FFFFFF"
    )
    style.configure("Treeview.Heading",
        background="#2C3E50",
        foreground="#FFFFFF"
    )
    
    # Set up columns
    columns = [f"Column {i+1}" for i in range(len(results[0]))]
    tree["columns"] = columns
    tree["show"] = "headings"
    
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=100)
    
    # Insert data
    for row in results:
        tree.insert("", "end", values=row)
    
    # Add scrollbar
    scrollbar = ttk.Scrollbar(frame, orient="vertical", command=tree.yview)
    tree.configure(yscrollcommand=scrollbar.set)
    
    # Pack everything
    tree.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")
# Color scheme constants
COLORS = {
    'bg_primary': '#f0f2f5',          # Light gray background
    'bg_secondary': '#e6f7ff',        # Light blue background
    'header_bg': '#003366',           # Dark blue header
    'text_dark': '#333333',           # Dark text
    'text_light': 'white',            # White text
    'button_primary': '#0073e6',      # Blue buttons
    'button_secondary': '#005bb5',    # Darker blue buttons
    'button_danger': '#d9534f',       # Red button
    'divider': '#bfbfbf'             # Gray divider
}

def show_main_window():
    global main_window
    if 'login_window' in globals():
        login_window.destroy()

    main_window = tk.Tk()
    main_window.title("Inventory Management System")
    main_window.geometry("1200x900")
    main_window.configure(bg="#FFFFFF")  # White background

    # Create main frame
    main_frame = tk.Frame(main_window, bg="#FFFFFF")
    main_frame.pack(fill='both', expand=True, padx=20, pady=20)

    # Header with dark background
    header_frame = tk.Frame(main_frame, bg="#2C3E50", pady=20)  # Dark blue header
    header_frame.pack(fill='x')

    title_label = tk.Label(
        header_frame,
        text="Inventory Management System",
        font=("Helvetica", 24, "bold"),
        bg="#2C3E50",  # Dark blue background
        fg="#FFFFFF"   # White text
    )
    title_label.pack()

    # Content frame
    content_frame = tk.Frame(main_frame, bg="#FFFFFF")
    content_frame.pack(fill='both', expand=True, pady=20)

    # CRUD Operations Section
    crud_label = tk.Label(
        content_frame,
        text="CRUD Operations",
        font=("Helvetica", 20, "bold"),
        bg="#FFFFFF",  # White background
        fg="#2C3E50"   # Dark blue text
    )
    crud_label.pack(pady=10)

    # CRUD Buttons
    crud_buttons = [
        ("Create Record", create_record),
        ("Read Records", read_records),
        ("Update Record", update_record),
        ("Delete Record", delete_record)
    ]

    for text, command in crud_buttons:
        btn = tk.Button(
            content_frame,
            text=text,
            command=command,
            font=("Helvetica", 12),
            bg="#3498DB",  # Blue background
            fg="#FFFFFF",  # White text
            pady=10,
            relief="raised",
            borderwidth=2
        )
        btn.pack(fill='x', padx=50, pady=5)

    # Divider
    tk.Frame(content_frame, height=2, bg="#BDC3C7").pack(fill='x', padx=50, pady=20)

    # Advanced Queries Section
    advanced_label = tk.Label(
        content_frame,
        text="Advanced SQL Queries",
        font=("Helvetica", 20, "bold"),
        bg="#FFFFFF",  # White background
        fg="#2C3E50"   # Dark blue text
    )
    advanced_label.pack(pady=10)

    # Advanced Query Buttons
    query_buttons = [
        ("Top 5 Best-Selling Products", top_selling_products),
        ("Monthly Sales Trend", monthly_sales_trend),
        ("Product Reorder Alert", product_reorder_alert),
        ("Customer Ranking", customer_ranking),
        ("Category Sales Analysis", category_sales_analysis)
    ]

    for text, command in query_buttons:
        btn = tk.Button(
            content_frame,
            text=text,
            command=command,
            font=("Helvetica", 12),
            bg="#2980B9",  # Darker blue background
            fg="#FFFFFF",  # White text
            pady=10,
            relief="raised",
            borderwidth=2
        )
        btn.pack(fill='x', padx=50, pady=5)

    # Exit Button
    exit_btn = tk.Button(
        content_frame,
        text="Exit",
        command=main_window.quit,
        font=("Helvetica", 12),
        bg="#E74C3C",  # Red background
        fg="#FFFFFF",  # White text
        pady=10,
        relief="raised",
        borderwidth=2
    )
    exit_btn.pack(pady=20)

    main_window.mainloop()
def show_login_window():
    try:
        global login_window
        login_window = tk.Tk()
        login_window.title("Login to Inventory Management System")
        login_window.geometry("500x400")
        login_window.configure(bg="white")  # White background for visibility

        # Header Frame
        header_frame = tk.Frame(login_window, bg="#2C3E50")  # Dark blue background
        header_frame.pack(fill="x", pady=20)

        # Title
        title_label = tk.Label(
            header_frame,
            text="Inventory Management System",
            font=("Arial", 20, "bold"),
            bg="#2C3E50",
            fg="white"
        )
        title_label.pack(pady=10)

        # Login Frame
        login_frame = tk.Frame(login_window, bg="white")
        login_frame.pack(pady=20, padx=40, fill="both", expand=True)

        # Username
        username_label = tk.Label(
            login_frame,
            text="Username:",
            font=("Arial", 12),
            bg="white",
            fg="#2C3E50"
        )
        username_label.pack(pady=5)
        
        username_entry = tk.Entry(
            login_frame,
            font=("Arial", 12)
        )
        username_entry.pack(pady=5)

        # Password
        password_label = tk.Label(
            login_frame,
            text="Password:",
            font=("Arial", 12),
            bg="white",
            fg="#2C3E50"
        )
        password_label.pack(pady=5)
        
        password_entry = tk.Entry(
            login_frame,
            font=("Arial", 12),
            show="*"
        )
        password_entry.pack(pady=5)

        # Login Button
        login_button = tk.Button(
            login_frame,
            text="Login",
            command=lambda: connect_to_database(username_entry.get(), password_entry.get()),
            font=("Arial", 12),
            bg="#3498DB",
            fg="white",
            width=20,
            height=2
        )
        login_button.pack(pady=20)

        login_window.mainloop()
        
    except Exception as e:
        print(f"Error in login window: {e}")
if __name__ == "__main__":
    try:
        show_login_window()
    except Exception as e:
        print(f"Error starting application: {e}")